# Ensemble Assesment - Boosting and Gradient Boosting

---
## Import modules

In [ ]:
import numpy as np # linear algebra
import pandas as pd  # data management (dataframes)
import matplotlib.pyplot as plt
import seaborn as sns  # plotting

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree, export_text
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

from sklearn.inspection import permutation_importance

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostRegressor, GradientBoostingRegressor
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import BaggingRegressor
#from sklearn.ensemble import RandomForestClassifier

# others
from mltools import classification_tools as CT
from mltools import model_tools as MT

---
## Load data and split into train and test sets
Data is loaded from the processed data file and split into train and test sets. The train set is used to train the model and the test set is used to evaluate the model.

In [ ]:
# Load data
file_path = 'data/input.csv'
df = pd.read_csv(file_path)
df.head(10)

,MES_x,DIA_x,HOUR,IRRAD_VALUE,UTIL_VALUE,dia_mes
0,1,1,0,0.0,0.000023,1
1,1,2,0,0.0,0.000012,2
2,1,3,0,0.0,0.000008,3
3,1,4,0,0.0,0.000008,4
4,1,5,0,0.0,0.000008,5
5,1,6,0,0.0,0.000012,6
6,1,7,0,0.0,0.000000,7
7,1,8,0,0.0,0.000000,8
8,1,9,0,0.0,0.000004,9
9,1,10,0,0.0,0.000000,10


In [ ]:
## Split the dataset into training and test sets

# Define input and output matrices
INPUTS = df.columns[~df.columns.isin(['UTIL_VALUE'])]
OUTPUT = 'UTIL_VALUE'

# Split
X_train, X_test, y_train, y_test = train_test_split(df[INPUTS], df[OUTPUT],
                                                    test_size=0.2, # percentage of test data
                                                    random_state=0) # seed for replication

# Check the shape of the resulting dataframes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (14028, 5), y_train shape: (14028,)
X_test shape: (3508, 5), y_test shape: (3508,)
